# MLP Hyperparameter Optimization

### Setting up variables, library and data

In [1]:
import pandas as pd
import numpy as np

## Data path handlers
TRAIN_TEST_SET_PATH = '../../Data/train_test/'
META_DATA_PATH = '../../Data/meta/'
AUG_SET_PATH = '../../Data/aug_data_all/'
MODEL_PATH = '../../Models/'
IMAGES_PATH = '../../Images/'

## Selecting the desired features
parameters = 'fz'

In [2]:
### Getting data
X_train = pd.read_csv(TRAIN_TEST_SET_PATH+'X_train.csv',index_col=0)
y_train = pd.read_csv(TRAIN_TEST_SET_PATH+'y_train.csv',index_col=0)
X_test = pd.read_csv(TRAIN_TEST_SET_PATH+'X_test.csv',index_col=0)
y_test = pd.read_csv(TRAIN_TEST_SET_PATH+'y_test.csv',index_col=0)

In [3]:
### Selecting features
X_train = X_train.iloc[:, X_train.columns.str.contains(parameters)]
X_test = X_test.iloc[:, X_test.columns.str.contains(parameters)]

X_train['labels'] = y_train.copy()

TypeError: first argument must be string or compiled pattern

In [4]:
### Creating the validation set
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=27)
for train, val in split.split(X_train, X_train['labels']):
    X_train_vl = X_train.iloc[train].copy()
    X_val = X_train.iloc[val].copy()
    
y_train_vl = X_train_vl['labels'].copy()
y_val = X_val['labels'].copy()


X_train_vl = X_train_vl.iloc[:, ~X_train_vl.columns.str.contains('labels')]
X_val = X_val.iloc[:, ~X_val.columns.str.contains('labels')]
X_train = X_train.iloc[:, ~X_train.columns.str.contains('labels')]

y_train_vl = np.array(y_train_vl)-1
y_val = np.array(y_val)-1
y_test = np.array(y_test)-1
y_train = np.array(y_train)-1

In [5]:
### Standardizing the data
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)
X_train_vl = std_scaler.transform(X_train_vl)
X_val = std_scaler.transform(X_val)

### Setting up the mlp model and callbacks

In [6]:
### Training a simple mlp to use as reference
import tensorflow as tf
from tensorflow import keras

##Model Sketch

In [7]:
### Model sketch
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[1556]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy'
                 ,optimizer=optimizer
                 ,metrics=['accuracy'])
    return model

mlp_simple = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [8]:
### Setting Callbacks
early_stopping = keras.callbacks.EarlyStopping(patience=15,
                                               restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint(MODEL_PATH+"mlp_simple.h5",
                                                save_best_only=True)                                     


### Hyperparameter optimization

In [9]:
### Setting parameters range for hyperparameter optimzation
### Here we will use random search as the optimization method
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV


params = {
    'n_hidden': np.arange(1,18),
    'n_neurons': np.arange(1,125),
    'learning_rate': reciprocal(3e-4, 3e-2),
    'input_shape': [X_train_vl.shape[1]]
}

In [10]:
### Hyperparameter Optimization
rnd_search = RandomizedSearchCV(mlp_simple, params, n_iter=10, cv=3, n_jobs=1)
rnd_search.fit(X_train_vl, y_train_vl, epochs=100,
               validation_split=0.15,
               callbacks=[early_stopping, checkpoint_cb])

Train on 171 samples, validate on 31 samples
Epoch 1/100
171/171 [==============================] - 2s 14ms/sample - loss: 1.0970 - accuracy: 0.4327 - val_loss: 1.0902 - val_accuracy: 0.7097
Epoch 2/100
171/171 [==============================] - 0s 3ms/sample - loss: 1.0900 - accuracy: 0.6082 - val_loss: 1.0815 - val_accuracy: 0.7097
Epoch 3/100
171/171 [==============================] - 0s 3ms/sample - loss: 1.0837 - accuracy: 0.6199 - val_loss: 1.0732 - val_accuracy: 0.7097
Epoch 4/100
171/171 [==============================] - 0s 2ms/sample - loss: 1.0777 - accuracy: 0.6199 - val_loss: 1.0658 - val_accuracy: 0.7097
Epoch 5/100
171/171 [==============================] - 0s 2ms/sample - loss: 1.0722 - accuracy: 0.6199 - val_loss: 1.0586 - val_accuracy: 0.7097
Epoch 6/100
171/171 [==============================] - 0s 3ms/sample - loss: 1.0668 - accuracy: 0.6199 - val_loss: 1.0517 - val_accuracy: 0.7097
Epoch 7/100
171/171 [==============================] - 0s 3ms/sample - loss: 1.0617 

Epoch 57/100
171/171 [==============================] - 0s 2ms/sample - loss: 0.9193 - accuracy: 0.6199 - val_loss: 0.8399 - val_accuracy: 0.7097
Epoch 58/100
171/171 [==============================] - 0s 2ms/sample - loss: 0.9178 - accuracy: 0.6199 - val_loss: 0.8368 - val_accuracy: 0.7097
Epoch 59/100
171/171 [==============================] - 0s 2ms/sample - loss: 0.9159 - accuracy: 0.6199 - val_loss: 0.8349 - val_accuracy: 0.7097
Epoch 60/100
171/171 [==============================] - 0s 2ms/sample - loss: 0.9148 - accuracy: 0.6199 - val_loss: 0.8321 - val_accuracy: 0.7097
Epoch 61/100
171/171 [==============================] - 0s 3ms/sample - loss: 0.9128 - accuracy: 0.6199 - val_loss: 0.8294 - val_accuracy: 0.7097
Epoch 62/100
171/171 [==============================] - 0s 2ms/sample - loss: 0.9112 - accuracy: 0.6199 - val_loss: 0.8266 - val_accuracy: 0.7097
Epoch 63/100
171/171 [==============================] - 0s 3ms/sample - loss: 0.9096 - accuracy: 0.6199 - val_loss: 0.8248 -

Epoch 12/100
172/172 [==============================] - 0s 1ms/sample - loss: 1.0269 - accuracy: 0.6453 - val_loss: 1.0075 - val_accuracy: 0.7097
Epoch 13/100
172/172 [==============================] - 0s 1ms/sample - loss: 1.0218 - accuracy: 0.6453 - val_loss: 1.0007 - val_accuracy: 0.7097
Epoch 14/100
172/172 [==============================] - 0s 1ms/sample - loss: 1.0163 - accuracy: 0.6453 - val_loss: 0.9942 - val_accuracy: 0.7097
Epoch 15/100
172/172 [==============================] - 0s 1ms/sample - loss: 1.0113 - accuracy: 0.6453 - val_loss: 0.9878 - val_accuracy: 0.7097
Epoch 16/100
172/172 [==============================] - 0s 1ms/sample - loss: 1.0064 - accuracy: 0.6453 - val_loss: 0.9820 - val_accuracy: 0.7097
Epoch 17/100
172/172 [==============================] - 0s 1ms/sample - loss: 1.0017 - accuracy: 0.6453 - val_loss: 0.9760 - val_accuracy: 0.7097
Epoch 18/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.9972 - accuracy: 0.6453 - val_loss: 0.9700 -

Epoch 23/100
172/172 [==============================] - 0s 843us/sample - loss: 0.9865 - accuracy: 0.6279 - val_loss: 0.9528 - val_accuracy: 0.6774
Epoch 24/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.9829 - accuracy: 0.6279 - val_loss: 0.9479 - val_accuracy: 0.6774
Epoch 25/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.9793 - accuracy: 0.6279 - val_loss: 0.9436 - val_accuracy: 0.6774
Epoch 26/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.9762 - accuracy: 0.6279 - val_loss: 0.9395 - val_accuracy: 0.6774
Epoch 27/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.9732 - accuracy: 0.6279 - val_loss: 0.9351 - val_accuracy: 0.6774
Epoch 28/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.9701 - accuracy: 0.6279 - val_loss: 0.9311 - val_accuracy: 0.6774
Epoch 29/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.9672 - accuracy: 0.6279 - val_loss: 0.9273

Epoch 79/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.8998 - accuracy: 0.6279 - val_loss: 0.8196 - val_accuracy: 0.6774
Epoch 80/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.8990 - accuracy: 0.6279 - val_loss: 0.8181 - val_accuracy: 0.6774
Epoch 81/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.8983 - accuracy: 0.6279 - val_loss: 0.8170 - val_accuracy: 0.6774
Epoch 82/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.8977 - accuracy: 0.6279 - val_loss: 0.8160 - val_accuracy: 0.6774
Epoch 83/100
172/172 [==============================] - 0s 2ms/sample - loss: 0.8972 - accuracy: 0.6279 - val_loss: 0.8149 - val_accuracy: 0.6774
Epoch 84/100
172/172 [==============================] - 0s 2ms/sample - loss: 0.8965 - accuracy: 0.6279 - val_loss: 0.8136 - val_accuracy: 0.6774
Epoch 85/100
172/172 [==============================] - 0s 2ms/sample - loss: 0.8959 - accuracy: 0.6279 - val_loss: 0.8128 -

...

172/172 [==============================] - 0s 976us/sample - loss: 0.7001 - accuracy: 0.6453 - val_loss: 0.6219 - val_accuracy: 0.7097
Epoch 48/100
172/172 [==============================] - 0s 824us/sample - loss: 0.6834 - accuracy: 0.6453 - val_loss: 0.6119 - val_accuracy: 0.7097
Epoch 49/100
172/172 [==============================] - 0s 753us/sample - loss: 0.6685 - accuracy: 0.6453 - val_loss: 0.6039 - val_accuracy: 0.7097
Epoch 50/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.6517 - accuracy: 0.6453 - val_loss: 0.5978 - val_accuracy: 0.7097
Epoch 51/100
172/172 [==============================] - 0s 861us/sample - loss: 0.6370 - accuracy: 0.6453 - val_loss: 0.5942 - val_accuracy: 0.7097
Epoch 52/100
172/172 [==============================] - 0s 856us/sample - loss: 0.6220 - accuracy: 0.6453 - val_loss: 0.5911 - val_accuracy: 0.7097
Epoch 53/100
172/172 [==============================] - 0s 880us/sample - loss: 0.6076 - accuracy: 0.6453 - val_loss: 0.5910 - 

Epoch 31/100
172/172 [==============================] - 0s 674us/sample - loss: 0.8891 - accuracy: 0.6279 - val_loss: 0.7985 - val_accuracy: 0.6774
Epoch 32/100
172/172 [==============================] - 0s 989us/sample - loss: 0.8862 - accuracy: 0.6279 - val_loss: 0.7946 - val_accuracy: 0.6774
Epoch 33/100
172/172 [==============================] - 0s 1ms/sample - loss: 0.8837 - accuracy: 0.6279 - val_loss: 0.7901 - val_accuracy: 0.6774
Epoch 34/100
172/172 [==============================] - 0s 754us/sample - loss: 0.8807 - accuracy: 0.6279 - val_loss: 0.7880 - val_accuracy: 0.6774
Epoch 35/100
172/172 [==============================] - 0s 891us/sample - loss: 0.8780 - accuracy: 0.6279 - val_loss: 0.7828 - val_accuracy: 0.6774
Epoch 36/100
172/172 [==============================] - 0s 771us/sample - loss: 0.8750 - accuracy: 0.6279 - val_loss: 0.7793 - val_accuracy: 0.6774
Epoch 37/100
172/172 [==============================] - 0s 834us/sample - loss: 0.8719 - accuracy: 0.6279 - val_lo

/home/segreto/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Train on 258 samples, validate on 46 samples
Epoch 1/100
258/258 [==============================] - 1s 4ms/sample - loss: 1.0448 - accuracy: 0.4380 - val_loss: 0.9190 - val_accuracy: 0.6957
Epoch 2/100
258/258 [==============================] - 0s 393us/sample - loss: 0.9236 - accuracy: 0.6085 - val_loss: 0.8524 - val_accuracy: 0.6957
Epoch 3/100
258/258 [==============================] - 0s 529us/sample - loss: 0.8507 - accuracy: 0.6357 - val_loss: 0.8116 - val_accuracy: 0.6957
Epoch 4/100
258/258 [==============================] - 0s 521us/sample - loss: 0.8032 - accuracy: 0.6744 - val_loss: 0.7753 - val_accuracy: 0.6957
Epoch 5/100
258/258 [==============================] - 0s 514us/sample - loss: 0.7586 - accuracy: 0.6860 - val_loss: 0.7406 - val_accuracy: 0.6957
Epoch 6/100
258/258 [==============================] - 0s 495us/sample - loss: 0.7195 - accuracy: 0.6938 - val_loss: 0.7213 - val_accuracy: 0.8043
Epoch 7/100
258/258 [==============================] - 0s 418us/sample - lo

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f5d5c12fc18>,
                   iid='warn', n_iter=10, n_jobs=1,
                   param_distributions={'input_shape': [1556],
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5d5c12f748>,
                                        'n_hidden': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,...
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124]

In [11]:
### Best validation score
rnd_search.best_score_

0.8289473656761018

In [13]:
### Best set of hyperparameters
best_parameters = pd.DataFrame(rnd_search.best_params_, index=['values'])
best_parameters.to_csv(MODEL_PATH+'mlp_simple_bp.csv')

rnd_search.best_params_

{'input_shape': 1556,
 'learning_rate': 0.007125723701969158,
 'n_hidden': 4,
 'n_neurons': 49}

In [30]:
pd.read_csv(MODEL_PATH+'mlp_simple_bp.csv', usecols=['learning_rate']).iloc[0]

learning_rate    0.007126
Name: 0, dtype: float64

### Training the reference model

In [14]:
### Selecting and retraining the best model
best = rnd_search.best_estimator_.model
best.fit(X_train, y_train, epochs=200,
         validation_data=(X_test, y_test),
         callbacks=[early_stopping])

Train on 380 samples, validate on 96 samples
Epoch 1/200
380/380 [==============================] - 0s 1ms/sample - loss: 0.3844 - accuracy: 0.8605 - val_loss: 0.6585 - val_accuracy: 0.8229
Epoch 2/200
380/380 [==============================] - 0s 458us/sample - loss: 0.3688 - accuracy: 0.8632 - val_loss: 0.6546 - val_accuracy: 0.8229
Epoch 3/200
380/380 [==============================] - 0s 420us/sample - loss: 0.3562 - accuracy: 0.8658 - val_loss: 0.6549 - val_accuracy: 0.8229
Epoch 4/200
380/380 [==============================] - 0s 576us/sample - loss: 0.3451 - accuracy: 0.8658 - val_loss: 0.6549 - val_accuracy: 0.8229
Epoch 5/200
380/380 [==============================] - 0s 764us/sample - loss: 0.3356 - accuracy: 0.8684 - val_loss: 0.6549 - val_accuracy: 0.8229
Epoch 6/200
380/380 [==============================] - 0s 576us/sample - loss: 0.3276 - accuracy: 0.8711 - val_loss: 0.6573 - val_accuracy: 0.8229
Epoch 7/200
380/380 [==============================] - 0s 751us/sample - lo

In [50]:
### Model evaluation in accuracy terms
best.evaluate(X_test, y_test)

96/96 [==============================] - 0s 592us/sample - loss: 0.6546 - accuracy: 0.8229


[0.6545854012171427, 0.8229167]